# Feature Engineering

## Review

1. Aquire
2. Prep
3. Split
4. Scale
5. Explore
6. Evaluation

## Feature Engineering

1. Construct new features: Use domain knowledge, creating products of features, etc. 
1. Use recursive feature elimination to recursively remove attributes to meet the number of required features and then builds a model on those attributes that remain to see if you can you match or improve performance with a smaller subset.   
1. Use backward elimination to recursively remove the worst performing features one by one till the overall performance of the model comes in acceptable range.
1. Forward selection begins with an empty equation. Predictors are added one at a time beginning with the predictor with the highest correlation with the dependent variable. Variables of greater theoretical importance are entered first. Once in the equation, the variable remains there.
1. Compare several of these feature selection methods. Select the best approach with model selection.
1. To improve performance and/or understanding, subsample your data and redo your analysis for several samples.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Acquire

### Prep

### Split

### Explore

1. are my independent variables independent of each other? (ideal)
2. is my dependent variable dependent on the independent variables? (ideal)
3. what are the drivers of my target variable? (could be some/all independent variables and/or other features not already identified)


#### Takeaway


### Construct new features

### Scale

### RFE: Recursive Feature Elimination

recursively remove attributes to meet the number of required features and then builds a model on those attributes that remain to see if you can you match or improve performance with a smaller subset.   


### Select K-Best
SelectKBest removes all but the highest scoring features
Scores are the test statistic for the chosen function or test (Chi-squared, F-regression, etc.)


### Wrapper Methods